In [1]:
# load original questions TODO
n = 5
qs = []

import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

with open('../annotated_wd_data_valid_answerable.txt') as fin:
    lines = fin.readlines()
    for l in lines[:n]:
#         subject [tab] property [tab] object [tab] question
        s, p, o, q = l.strip('\n').split('\t')
        q = re.sub(chars_to_ignore_regex, '', q).lower()
        print(q)
        qs.append(q)

where was sasha vujačić born
what is a film directed by wiebke von carolsfeld
what was seymour parker gilbert's profession
in what french city did antoine de févin die 
what job does jamie hewlett have


In [14]:
# ASR FB_S2T
# for i in *.wav; do ffmpeg -i $i -ar 1600 -acodec pcm_s16le -ac 1 ./mono/$(basename $i); done
import os
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from datasets import load_dataset
import soundfile as sf

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

path = "../dev/"

# init transcripts
ts = [''] * n

# model.to("cuda")

for file in os.listdir(path):
    speech, samplerate = sf.read(path+file)
    i = int(file.split('.')[0]) - 1
    
    input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)
    ts[i] = transcription[0].lower()

for t in ts:
    print(t)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


where was sacha buyachich born
what is a hume directed by vipcafon caronsfelt
what was say more parker gilbert's profession
in what french seaty did antoinne de fevinnedaie
what job does jamie he let have


In [15]:
#### calculate metrics TODO
for i, t in enumerate(ts):
    print(qs[i], t)
    
from datasets import load_metric
wer_metric = load_metric("wer")
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=ts, references=qs)))

wer_metric = load_metric("cer")

print("Test CER: {:.3f}".format(wer_metric.compute(predictions=ts, references=qs)))

where was sasha vujačić born where was sacha buyachich born
what is a film directed by wiebke von carolsfeld what is a hume directed by vipcafon caronsfelt
what was seymour parker gilbert's profession what was say more parker gilbert's profession
in what french city did antoine de févin die  in what french seaty did antoinne de fevinnedaie
what job does jamie hewlett have what job does jamie he let have
Test WER: 0.400
Test CER: 0.173
